In [117]:
import requests
import base64
import os
import json

In [83]:
OAUTH_TOKEN = 'y0_AgAAAAAMexrgAATuwQAAAADVqdt6kU7XLevzTjaytB5Gybh3qHB8uBI'     # OAuth токен аккаунта Яндекс
FOLDER_ID = 'b1gjjshh8f4c2hsghon5'       # Идентификатор рабочего каталога Я.Cloud
WORK_DIR = 'pass_photos'
GET_AIM_TOKEN_URL = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
GET_IMAGE_RECOGNITION_URL = 'https://vision.api.cloud.yandex.net/vision/v1/batchAnalyze'

### Обмениваем OAuth-токен аккаунта Яндекс на IAM-токен

In [85]:
data = {
    'yandexPassportOauthToken': OAUTH_TOKEN
}
 
response = requests.post(url=GET_AIM_TOKEN_URL, json=data)
print(response)
iam_token = response.json()['iamToken']

<Response [200]>


### Вынес некоторый код в функции для удобства, чтобы потом в цикле пройтись по всем изображениям

In [87]:
# Функция на основе инструкции Яндекс.Cloud
def encode_file(file_name: str):             # Не знаю как правильно обозначить тип возвращаемого значения
    with open (file_name, 'rb') as f:        # Яндекс забыл про параметр 'rb', без которого невозможно прочитать изображение
        file_content = f.read()
        return base64.b64encode(file_content)    

In [88]:
# Функция для получения списка файлов из каталога с фотографиями (как в Task 1)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [106]:
files = get_files(WORK_DIR)
headers = {'Authorization': f'Bearer {iam_token}', 'Content-Type': 'application/json'}

# print(encode_file(files[4]))

# проходим по всем изображениям, получаем для каждого представление в byte64 и отправляем запрос
# for file in files:



body_json = {
    "folderId": FOLDER_ID,
    "analyze_specs": [{
        "content": str(encode_file(files[0]))[2:-1],  # про то, что byte64 невозможно сериализовать в JSON они тоже забыли
        "features": [{
            "type": "TEXT_DETECTION",
            "text_detection_config": {
                "language_codes": ["ru"],
                "model": "passport"
            }
        }]
    }]
}

response = requests.post(url=GET_IMAGE_RECOGNITION_URL, headers=headers, json=body_json)
print(response)
print(response.json())

<Response [200]>
{'results': [{'results': [{'textDetection': {'pages': [{'blocks': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'lines': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'words': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'languages': [{'languageCode': 'en', 'confidence': 0.8926920295}], 'text': 'rus', 'confidence': 0.8926920295}], 'confidence': 0.8926920295}]}, {'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'lines': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'words': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'languages': [{'languageCode': 'en', 'confidence': 0.8926920295}], 'text': '-', 'confidence': 0.8926920295, 'entityIndex': '1'}], 'confidence': 0.8926920295}]}, {'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'lines': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'words': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'languages': [{'languageCode': 'en', 'confidence': 0.7420556545}], 'text': 'муж', 'confidence': 0.7420556545, 'entityInd

In [126]:
print(response.json()['results'][0]['results'][0]['textDetection']['pages'][0]['blocks'])
with open('task_2.json', 'w') as f:
    # response.json()['results'] = base64.encodebytes(content.encode('cp1251')).decode('ascii')
    json.dump(response.json(), f, indent=4, ensure_ascii=False)

[{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'lines': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'words': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'languages': [{'languageCode': 'en', 'confidence': 0.8926920295}], 'text': 'rus', 'confidence': 0.8926920295}], 'confidence': 0.8926920295}]}, {'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'lines': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'words': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'languages': [{'languageCode': 'en', 'confidence': 0.8926920295}], 'text': '-', 'confidence': 0.8926920295, 'entityIndex': '1'}], 'confidence': 0.8926920295}]}, {'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'lines': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'words': [{'boundingBox': {'vertices': [{}, {}, {}, {}]}, 'languages': [{'languageCode': 'en', 'confidence': 0.7420556545}], 'text': 'муж', 'confidence': 0.7420556545, 'entityIndex': '2'}], 'confidence': 0.7420556545}]}, {'boundingBox': {'vertices': [{'x': '74'

In [148]:
pass_data = response.json()['results'][0]['results'][0]['textDetection']['pages'][0]['entities']
print(pass_data)

issued_by = pass_data[4]['text']    # Кем выдан
expiration_date = pass_data[1]['text']    # Дата выдачи
subdivision = pass_data[3]['text']         # Код подразделения
surname = pass_data[5]['text'] # Фамилия
name = pass_data[6]['text'] # Имя
middle_name = pass_data[7]['text'] # Отчество
gender = pass_data[2]['text'] # Пол
birth_date = pass_data[8]['text'] # Дата рождения
birth_place = pass_data[9]['text']# Место рождения
number = pass_data[10]['text'] # Номер паспорта

result_text = f'''Кем выдан: {issued_by}\nДата выдачи: {expiration_date}\nКод подразделения: {subdivision}\n
Номер паспорта: {number}\nФамилия: {surname}\nИмя: {name}\nОтчество: {middle_name}\nПол: {gender}\n
Дата рождения: {birth_date}\nМесто Рождения: {birth_place}\n'''
print(result_text)

[{'name': 'citizenship', 'text': 'rus'}, {'name': 'expiration_date', 'text': '-'}, {'name': 'gender', 'text': 'муж'}, {'name': 'subdivision', 'text': '772-112'}, {'name': 'issued_by', 'text': 'паспортно-визовым отделением овд пресненского района увд цао г. москвы'}, {'name': 'surname', 'text': 'трамп'}, {'name': 'name', 'text': 'дональд'}, {'name': 'middle_name', 'text': 'джон'}, {'name': 'birth_date', 'text': '14.06.1946'}, {'name': 'birth_place', 'text': 'гор. куинс штат нью-йорк сша'}, {'name': 'number', 'text': '4507123456'}]
Кем выдан: паспортно-визовым отделением овд пресненского района увд цао г. москвы
Дата выдачи: -
Код подразделения: 772-112

Номер паспорта: 4507123456
Фамилия: трамп
Имя: дональд
Отчество: джон
Пол: муж

Дата рождения: 14.06.1946
Место Рождения: гор. куинс штат нью-йорк сша

